# Library Manager.ipynb
 
The Library Manager Notebook collects the commands for managing the reference library.  The library is a local copy of records from https://potentials.nist.gov/. iprPy uses the [potentials](https://github.com/usnistgov/potentials) Python package to interact with these records, and as such, all settings and downloaded reference files are common for iprPy, potentials and atomman.

The included tools in this Notebook allow for

- Defining the library settings,
- Downloading reference files from https://potentials.nist.gov/, and
- Downloading additional reference records from [Materials Project](https://materialsproject.org/) and [OQMD](http://oqmd.org/).

In [1]:
from pathlib import Path

# https://github.com/usnistgov/iprPy
import iprPy
print('iprPy version', iprPy.__version__)

settings = iprPy.Settings()

iprPy version 0.10.2


## 1. Basic library settings

The iprPy.Settings class is a child of the potentials.Settings class and can be used to save user parameters for use with potentials, atomman, and iprPy.  All three packages share the same settings file as it is expected that there be only one local copy of the reference records from the potentials database.

See the [potentials](https://github.com/usnistgov/potentials) Python package for more details.

### 1.1. Settings directory

By default, the settings __directory__ is placed in the user's home directory.  It can be useful to change this to share settings between

- Tools that access the OS-defined "HOME" and platforms like Cygwin, MSYS2 that define their own "home" locations, and
- Multiple users on the same hardware.  Note: read/write access to the files would have to be managed elsewhere.

In [2]:
print('Default settings directory:', settings.defaultdirectory)
print('Current settings directory:', settings.directory)

Default settings directory: C:\Users\lmh1\.NISTpotentials
Current settings directory: C:\Users\lmh1\.NISTpotentials


In [3]:
#settings.set_directory(newpath)
#print('Current settings directory:', settings.directory)

### 1.2. Library directory

The __library_directory__ is the local path to the location where the reference records are copied to.  By default, this is placed in the settings directory.  As the default settings directory is in a "." folder, it is highly suggested to change the library directory to somewhere more accessible if you wish to manually explore the records.

In [4]:
print('Current library directory:', settings.library_directory)

Current library directory: C:\Users\lmh1\Documents\library


In [5]:
#settings.set_library_directory(newpath)
#print('Current library directory:', settings.library_directory)

## 2. Download reference records from potentials.nist.gov

Records from the potentials.nist.gov database can then be downloaded using the iprPy.Library class.  The class is a child of am.library.Database, which in turn is a child of potentials.Database giving it built-in tools for interacting with and exploring a variety of existing reference record styles.

Create a Library object.  Currently requires log in information to access records, but settings will be changed soon...

In [6]:
# Load library and sign in
with open('E:/potentials_nist_gov/password.txt') as f:
    user, pswd = f.read().strip().split()
library = iprPy.Library(username=user, password=pswd)

The Library.download_refs() method will access records from potentials.nist.gov and download them to the local library.  Only records that have been added/changed will be saved.

__NOTE__: Local records will be overwritten if a remote record with the same name and different content is found.  If you make changes to the local records that you would like to keep be sure to change the file's name to something unique.

Parameters

- __style__ (*str or list, optional*) The reference style(s) to download.  If not given, all reference style will be downloaded.
- __status__ (*str, list or None, optional*) Only the potential_LAMMPS records with the given status(es) will be downloaded.  Allowed values are 'active' (default), 'superseded', and 'retracted'.  If set to None, all hosted potential_LAMMPS will be downloaded.
- __format__ (*str, optional*) Indicates the file format to save the refs as: 'json' or 'xml'. Default is 'json'.
- __indent__ (*int or None, optional*) If a number, then the saved content will be indented making it easier to read but larger files.  If None, the files will be compact.  Default value is 4.
- __verbose__ (*bool, optional*) If True, informative messages will be printed.

In [7]:
# Specify the ref styles to download
style = [
    'crystal_prototype',
    'dislocation',
    'free_surface',
    'point_defect',
    'stacking_fault',
    'potential_LAMMPS',
]

# Specify the potential_LAMMPS status(es) to download
#status = 'active'  # Only the current versions
status = None      # All records even old and retracted versions

library.download_refs(style=style, status=status, verbose=True)

Found 19 of crystal_prototype
 - 19 duplicate records skipped
Found 6 of dislocation
 - 6 duplicate records skipped
Found 146 of free_surface
 - 146 duplicate records skipped
Found 38 of point_defect
 - 38 duplicate records skipped
Found 11 of stacking_fault
 - 11 duplicate records skipped
332 LAMMPS potentials saved to localpath
 - 332 duplicate potentials skipped


## 3. Download DFT crystal structures

The Library class also has methods for automatically downloading DFT crystal structures from Materials Project and OQMD into the local library. 

### 3.1. Define elements_set

The methods explore records in the DFT databases based on lists of elements.  Some examples for the values:

- ['Al'] will fetch only elemental Al structures
- ['Al', 'Ni'] will fetch elemental Al and Ni structures and Al-Ni compounds.

#### Option #1: Auto generate

This generates lists of elements based on the potentials in the database.  If potential_kwargs is blank, then all element combinations found in the potentials will be searched.

In [8]:
# Specify the potential record style
potential_record_style = 'potential_LAMMPS'

# Specify any potential delimiters
potential_kwargs = {}
#potential_kwargs['id'] = '1986--Foiles-S-M--Ag--LAMMPS--ipr1'
#potential_kwargs['pair_style'] = 

In [9]:
elements_set = set()
for potential in library.get_lammps_potentials(**potential_kwargs):
    elements_set.add(' '.join(sorted(potential.elements())))

elements_lists = []
for e in sorted(list(elements_set)):
    elements_lists.append(e.split())
elements_lists

[['Ag'],
 ['Ag', 'Au', 'Cu'],
 ['Ag', 'Au', 'Cu', 'Ni', 'Pd', 'Pt'],
 ['Ag', 'Cu'],
 ['Ag', 'H', 'Pd'],
 ['Ag', 'Ni'],
 ['Ag', 'O', 'Ta'],
 ['Al'],
 ['Al', 'Au', 'Si'],
 ['Al', 'Co'],
 ['Al', 'Co', 'Ni'],
 ['Al', 'Cu'],
 ['Al', 'Cu', 'Fe', 'Mg', 'Si'],
 ['Al', 'Cu', 'H'],
 ['Al', 'Fe'],
 ['Al', 'H', 'Ni'],
 ['Al', 'Mg'],
 ['Al', 'Mg', 'Zn'],
 ['Al', 'Mn', 'Pd'],
 ['Al', 'Nb', 'Ti'],
 ['Al', 'Ni'],
 ['Al', 'Ni', 'O'],
 ['Al', 'O'],
 ['Al', 'Pb'],
 ['Al', 'Sm'],
 ['Al', 'Ti'],
 ['Al', 'U'],
 ['As', 'Ga'],
 ['Au'],
 ['Au', 'Pt'],
 ['Au', 'Rh'],
 ['Au', 'Si'],
 ['B', 'C', 'N'],
 ['B', 'N'],
 ['Be'],
 ['Be', 'O'],
 ['Br', 'Cl', 'Cs', 'F', 'I', 'K', 'Li', 'Na', 'Rb'],
 ['C'],
 ['C', 'Cu'],
 ['C', 'Fe'],
 ['C', 'Fe', 'Mn', 'Si'],
 ['C', 'Fe', 'Ti'],
 ['C', 'H'],
 ['C', 'H', 'O'],
 ['C', 'Si'],
 ['Cd', 'Hg', 'S', 'Se', 'Te', 'Zn'],
 ['Cd', 'Se', 'Te'],
 ['Cd', 'Te'],
 ['Cd', 'Te', 'Zn'],
 ['Ce', 'O'],
 ['Co'],
 ['Co', 'Cr', 'Cu', 'Fe', 'Ni'],
 ['Co', 'Ni'],
 ['Cr'],
 ['Cr', 'Fe'],
 ['Cr', 'Fe'

#### Option #2: Manually define

In [10]:
#elements_lists = []
#elements_lists.append(['Fe'])
#elements_lists.append(['Cu'])
#elements_lists.append(['Ni'])
#elements_lists.append(['Cu', 'Ni'])

### 3.2. Fetch Materials Project reference structures

Accessing Materials Project requires an API key.  Here, the API key used is read in from an external file.

In [11]:
api_key_location = 'E:/Materials Project/API key.txt'
with open(api_key_location) as f:
    api_key = f.read()

for elements in elements_lists:
    print('-'.join(elements))
    try:
        library.download_mp_crystals(elements, api_key=api_key)
    except:
        print('Failed!')

Ag
Ag-Au-Cu
Ag-Au-Cu-Ni-Pd-Pt
Ag-Cu
Ag-H-Pd
Ag-Ni
Ag-O-Ta
Added mvc-388
Added mvc-13894
Al
Al-Au-Si
Al-Co
Al-Co-Ni
Al-Cu
Al-Cu-Fe-Mg-Si



Al-Cu-H
Al-Fe
Al-H-Ni
Al-Mg
Al-Mg-Zn
Al-Mn-Pd
Al-Nb-Ti
Al-Ni
Al-Ni-O
Added mvc-11155
Added mvc-11911
Added mvc-12893


C:\Users\lmh1\AppData\Local\Continuum\anaconda3\lib\site-packages\pymatgen\io\cif.py:1116: UserWarning: Issues encountered while parsing CIF: Some fractional co-ordinates rounded to ideal values to avoid issues with finite precision.
  warnings.warn("Issues encountered while parsing CIF: %s" % "\n".join(self.warnings))


Added mvc-14043
Added mvc-14277
Added mvc-16800
Added mvc-3776
Added mvc-4622
Added mvc-10289
Added mvc-12050
Added mvc-12901
Added mvc-13696
Added mvc-2316
Added mvc-2602
Added mvc-5627
Added mvc-6477
Added mvc-6822
Added mvc-6929
Added mvc-6939
Added mvc-894
Al-O
Al-Pb
Al-Sm
Al-Ti
Al-U
As-Ga
Au
Au-Pt
Au-Rh
Au-Si
B-C-N
B-N
Be
Be-O
Br-Cl-Cs-F-I-K-Li-Na-Rb
C
C-Cu
C-Fe
C-Fe-Mn-Si
C-Fe-Ti
C-H
C-H-O
C-Si
Cd-Hg-S-Se-Te-Zn
Cd-Se-Te
Cd-Te
Cd-Te-Zn
Ce-O
Co
Co-Cr-Cu-Fe-Ni
Co-Ni
Cr
Cr-Fe
Cr-Fe-Ni
Cr-Fe-W
Cr-Ni
Cs
Cu
Cu-Fe-Ni
Cu-H
Cu-Ni
Cu-Pb
Cu-Ta
Cu-Zr
Fe
Fe-Ni
Fe-O
Added mvc-10966
Added mvc-11541
Added mvc-11993
Added mvc-11999
Added mvc-12005
Added mvc-12039
Added mvc-12063
Added mvc-12125
Added mvc-12204
Added mvc-12205
Added mvc-12905
Added mvc-13181
Added mvc-13234
Added mvc-14925
Added mvc-15135
Added mvc-5967
Fe-P
Fe-V
Fe-W
Ga-In-N
Ga-N
Ge
Ge-Si
H-He-W
H-Mg
H-Pd
He-Ta
In-P
K
Li
Li-S
Mg
Mo
Mo-U
Mo-U-Xe
N-U
Na
Nb
Nb-Ni
Nb-Zr
Ni
Ni-Pd
Ni-Ti
Ni-Ti-V
Ni-Zr
O-Si
O-Ti
Added mvc-11423
Added mvc-

### 3.3. Fetch OQMD reference structures

Accessing Materials Project requires an API key.  Here, the API key used is read in from an external file.

In [12]:
for elements in elements_lists:
    print('-'.join(elements))
    try:
        library.download_oqmd_crystals(elements)
    except:
        print('Failed!')

Ag
Ag-Au-Cu
Failed!
Ag-Au-Cu-Ni-Pd-Pt
Failed!
Ag-Cu
Ag-H-Pd
Failed!
Ag-Ni
Ag-O-Ta
Failed!
Al
Al-Au-Si
Failed!
Al-Co
Al-Co-Ni
Failed!
Al-Cu
Al-Cu-Fe-Mg-Si
Failed!
Al-Cu-H
Failed!
Al-Fe
Al-H-Ni
Failed!
Al-Mg
Failed!
Al-Mg-Zn
Failed!
Al-Mn-Pd
Failed!
Al-Nb-Ti
Failed!
Al-Ni
Al-Ni-O
Failed!
Al-O
Failed!
Al-Pb
Al-Sm
Al-Ti
Failed!
Al-U
As-Ga
Au
Au-Pt
Au-Rh
Au-Si
B-C-N
Failed!
B-N
Be
Be-O
Br-Cl-Cs-F-I-K-Li-Na-Rb
C
Failed!
C-Cu
Failed!
C-Fe
Failed!
C-Fe-Mn-Si
Failed!
C-Fe-Ti
Failed!
C-H
Failed!
C-H-O
Failed!
C-Si
Failed!
Cd-Hg-S-Se-Te-Zn
Failed!
Cd-Se-Te
Failed!
Cd-Te
Cd-Te-Zn
Failed!
Ce-O
Failed!
Co
Co-Cr-Cu-Fe-Ni
Failed!
Co-Ni
Cr
Cr-Fe
Cr-Fe-Ni
Failed!
Cr-Fe-W
Failed!
Cr-Ni
Cs
Cu
Cu-Fe-Ni
Failed!
Cu-H
Cu-Ni
Cu-Pb
Cu-Ta
Cu-Zr
Failed!
Fe
Fe-Ni
Fe-O
Failed!
Fe-P
Fe-V
Fe-W
Ga-In-N
Failed!
Ga-N
Ge
Ge-Si
H-He-W
H-Mg
Failed!
H-Pd
He-Ta
In-P
K
Li
Li-S
Mg
Failed!
Mo
Mo-U
Mo-U-Xe
N-U
Na
Nb
Nb-Ni
Nb-Zr
Failed!
Ni
Ni-Pd
Ni-Ti
Failed!
Ni-Ti-V
Failed!
Ni-Zr
Failed!
O-Si
Failed!
O-Ti
Failed!
Pb
Pb-Sn
Pd
Pt
